### imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

### Data opening

In [2]:
data = pd.read_csv('data/data_after_cleaning_norm_dummy.csv')

In [3]:
# Get top 50 features XGBOOST

with open("C://Users//lexfo//PycharmProjects//thesis_lex//logs//log_lex//ETHBTC//results_XGBoost_AUC_345.txt",'r') as file:
    xg_res = file.readlines()
    xgboost_ranking = []
    for i in xg_res:
        xgboost_ranking.append(i[:-1])
top_50_XGBoost_ranking = xgboost_ranking[:50]

In [4]:
# Get top 50 features of XGBOOST after combining with multicol

with open("C://Users//lexfo//PycharmProjects//thesis_lex//results//Combining//XGBoost_and_Clustering.txt",'r') as file:
    xg_res = file.readlines()
    xgboost_and_clustering = []
    for i in xg_res:
        xgboost_and_clustering.append(i[:-1])
top_50_xgboost_and_clustering = xgboost_and_clustering[:50]

### create new dataset with top 50 featuers (and dummy)

In [5]:
nieuwe_dataset = pd.DataFrame()


In [6]:
for i in top_50_xgboost_and_clustering:
    nieuwe_dataset[i] = data[i]

In [7]:
nieuwe_dataset['dummy_next_start_time'] = data['dummy_next_start_time']

In [8]:
nieuwe_dataset = nieuwe_dataset[:50000]

In [9]:
columns = list(nieuwe_dataset.columns)
columns.append("dummy_next_start_time") 
final_columns = [column for column in columns if column!="dummy_next_start_time"]
X = nieuwe_dataset[final_columns].values
y = nieuwe_dataset["dummy_next_start_time"].values

In [10]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(X, y, 
                                                                            test_size = 0.25, 
                                                                            random_state = 42)

# random forest optimize


### create gridsearch & random search parameters

In [ ]:
random_grid = {  
                 'bootstrap': [True, False],
                 'max_depth': [10, 20, 40, 60, None],
                 'max_features': ['auto', 'sqrt'],
                 'min_samples_leaf': [1, 2, 4],
                 'min_samples_split': [2, 5, 10],
                 'n_estimators': [200, 400, 800, 1600, 2000]
}


param_grid = {
                'max_depth': [45, 85, None],
                'bootstrap': [True, False],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [1, 4, 6],
                'min_samples_split': [7, 15, 25],
                'n_estimators': [100, 1000, 2000, 3000]
}

In [ ]:
rf = RandomForestClassifier()

### try random search

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf,
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)

rf_random.fit(train_features, train_labels)

In [ ]:
rf_random.best_params_

In [ ]:
model_rfc = RandomForestClassifier(    n_estimators =600,
                                       max_depth=30,
                                       min_samples_split=5,
                                       min_samples_leaf=4,
                                       max_features='auto',
                                       bootstrap=True,
                                       n_jobs=-1)

In [ ]:
model_rfc.fit(train_features, train_labels)

In [ ]:
rfc1 = RandomForestClassifier(min_samples_leaf = rf_random.best_params_['min_samples_leaf'],
                              min_samples_split = rf_random.best_params_['min_samples_split'],
                              max_features=rf_random.best_params_['max_features'], 
                              n_estimators= rf_random.best_params_['n_estimators'], 
                              max_depth=rf_random.best_params_['max_depth'])

In [ ]:
rfc1.fit(train_features, train_labels)

In [ ]:
pred = rfc1.predict(test_features)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(test_labels,pred))

### Try grid search

In [ ]:
rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, 
                           param_grid = param_grid, 
                           cv = 3, 
                           n_jobs = -1, 
                           verbose = 2, 
                           return_train_score=True)

grid_search.fit(train_features, train_labels);

In [ ]:
grid_search.best_params_

In [ ]:
rfc2 = RandomForestClassifier(random_state=42,
                              min_samples_leaf = grid_search.best_params_['min_samples_leaf'],
                              min_samples_split = grid_search.best_params_['min_samples_split'],
                              max_features=grid_search.best_params_['max_features'], 
                              n_estimators= grid_search.best_params_['n_estimators'], 
                              max_depth=grid_search.best_params_['max_depth']
                             )

In [ ]:
rfc2.fit(train_features, train_labels)

In [ ]:
pred = rfc2.predict(test_features)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(test_labels,pred))

<br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br>

# SVM

In [11]:
param_grid = {'C':[0.001,0.001,0.01,1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear']}

In [12]:
grid = GridSearchCV(SVC(),param_grid,refit = True, n_jobs = 8, verbose=2)


In [13]:
grid.fit(train_features,train_labels)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 11.6min
[Parallel(n_jobs=8)]: Done 140 out of 140 | elapsed: 211.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [0.001, 0.001, 0.01, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [14]:
grid.best_params_

{'C': 1000, 'gamma': 1, 'kernel': 'linear'}